In [7]:
import torch
from torch import nn
from torchvision import transforms
from torchvision import datasets
from torch.utils.data import DataLoader
import torch.optim as optim
from torch.utils.data import Dataset

In [2]:
#batch_size的大小是64
batch_size = 64
#用Compose组合多个transform操作
transform = transforms.Compose([
    #ToTensor将图像中的字节转换成tensor;
    transforms.ToTensor(),
    #Normalize将数据进行标准化，1是均值，2是标准差
    transforms.Normalize((0.1307, ),(0.3081, ))
])

In [3]:

#训练数据集的MNIST的数据集的根在mnist文件夹中
train_dataset = datasets.MNIST(root='../dataset/mnist',
                              #让训练等于真，提取训练集
                              train=True,
                              #下载等于真，下载数据集
                              download=True,
                              #transform直接应用上面的transform
                              transform=transform) 
#训练加载器是数据加载器，引入训练数据集
train_loader = DataLoader(train_dataset,
                         #shuffle为真，将数据打乱
                         shuffle=True,
                         #batch的大小为前面设定好的大小
                         batch_size=batch_size)


#测试数据集的MNIST的数据集的根在mnist文件夹中
test_dataset = datasets.MNIST(root='../dataset/mnist/',
                             #训练为假，测试为真
                             train=False,
                             #下载为真，就是下载
                             download=True,
                             #transform直接应用上面的transform
                             transform=transform)
#测试加载器是数据加载器，引入测试数据集
test_loader = DataLoader(test_dataset,
                        #shuffle为假，不将数据打乱
                        shuffle=False,
                        #batch的大小为前面设定好的大小 
                        batch_size=batch_size)

  0%|          | 0/9912422 [00:00<?, ?it/s]

Extracting ../dataset/mnist\MNIST\raw\train-images-idx3-ubyte.gz to ../dataset/mnist\MNIST\raw



  0%|          | 0/28881 [00:00<?, ?it/s]

Extracting ../dataset/mnist\MNIST\raw\train-labels-idx1-ubyte.gz to ../dataset/mnist\MNIST\raw



  0%|          | 0/1648877 [00:00<?, ?it/s]

Extracting ../dataset/mnist\MNIST\raw\t10k-images-idx3-ubyte.gz to ../dataset/mnist\MNIST\raw



  0%|          | 0/4542 [00:00<?, ?it/s]

Extracting ../dataset/mnist\MNIST\raw\t10k-labels-idx1-ubyte.gz to ../dataset/mnist\MNIST\raw



In [13]:
#设Net为一个类，继承自Module模块
class Net(torch.nn.Module):
    #自身初始化
    def __init__(self):
        #调用父类初始化器
        super(Net,self).__init__()
        #线性变换将784变成10
        self.l1 = torch.nn.Linear(784,512)
        self.l2 = torch.nn.Linear(512,256)
        self.l3 = torch.nn.Linear(256,128)
        self.l4 = torch.nn.Linear(128,64)
        self.l5 = torch.nn.Linear(64,10)
    #定义自身前馈计算
    def forward(self,x):
        #用view改变x的形状，列为784，行自动计算得到
        x = x.view(-1,784)
        #用relu激活自身的线性函数
        x = torch.relu(self.l1(x))
        x = torch.relu(self.l2(x))
        x = torch.relu(self.l3(x))
        x = torch.relu(self.l4(x))
        #返回第五层，不激活最后一层,要接入softmax
        return self.l5(x)

In [14]:
net = Net()

In [15]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr = 0.01, momentum = 0.5)

In [22]:
#定义训练过程，epoch是循环次数
def train(epoch):
    #设初始损失值是0
    running_loss = 0.0
    #batch_idx表示进行多少次的batch的迭代，data是数据，用列举的方法将train_loader的数据提取出来，从0开始
    for batch_idx, data in enumerate(train_loader):
        inputs, target = data
        optimizer.zero_grad()
        loss = criterion(net(inputs), target)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
        if batch_idx % 300 == 299:
            print('{},{} loss:{:.6f}'.format(epoch + 1,batch_idx + 1,running_loss / 300))
            #将损失归零
            running_loss = 0.0
            print('{},{} loss:{:.6f}'.format(epoch + 1,batch_idx + 1,running_loss / 2000))

In [27]:
#定义test模块
def test():
    #正确的数量初始为0
    correct = 0
    #总数为0
    total = 0
    #用with方式让torch不计算梯度
    with torch.no_grad():
        #for循环data在测试加载器中
        for data in test_loader:
            #测试集里面的images相当于输入，输入和标签送到数据里面
            images,labels = data
            #将输入代入模型，求输出
            outputs = net(images)
            #用torch中的max函数，沿着输出的数据的维度为1方式，找到最大值和最大值的下标，
            #_是最大值，predicted是最大值的下标
            _, predicted = torch.max(outputs.data,dim=1)
            #总数等于标签的size取第0个元素相加求得总数
            total += labels.size(0)
            #正确是预测和标签相等标量求和
            correct += (predicted == labels).sum().item()
    #输出%d代表整数，%%代表%，%格式化输出得连接符号，正确除总数乘100
    print('Accuracy on test set: {}%'.format(100 * correct / total))  


In [28]:
for epoch in range(10): 
    #训练
    train(epoch)
    #测试
    test()

1,300 loss:0.014279
1,300 loss:0.000000
1,600 loss:0.018465
1,600 loss:0.000000
1,900 loss:0.020775
1,900 loss:0.000000
Accuracy on test set: 97.82%
2,300 loss:0.012600
2,300 loss:0.000000
2,600 loss:0.015676
2,600 loss:0.000000
2,900 loss:0.013459
2,900 loss:0.000000
Accuracy on test set: 97.86%
3,300 loss:0.009450
3,300 loss:0.000000
3,600 loss:0.012237
3,600 loss:0.000000
3,900 loss:0.012252
3,900 loss:0.000000
Accuracy on test set: 97.82%
4,300 loss:0.007301
4,300 loss:0.000000
4,600 loss:0.007666
4,600 loss:0.000000
4,900 loss:0.008440
4,900 loss:0.000000
Accuracy on test set: 97.95%
5,300 loss:0.004747
5,300 loss:0.000000
5,600 loss:0.007081
5,600 loss:0.000000
5,900 loss:0.006555
5,900 loss:0.000000
Accuracy on test set: 97.96%
6,300 loss:0.004463
6,300 loss:0.000000
6,600 loss:0.003598
6,600 loss:0.000000
6,900 loss:0.004775
6,900 loss:0.000000
Accuracy on test set: 97.93%
7,300 loss:0.003543
7,300 loss:0.000000
7,600 loss:0.003518
7,600 loss:0.000000
7,900 loss:0.003276
7,900 